In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from typing import TypedDict


In [2]:
load_dotenv()

True

In [3]:
model=ChatOpenAI()

In [9]:
class BLOGState(TypedDict):
    title: str
    outline: str
    blog: str
    score: int

In [10]:
outline_prompt= PromptTemplate(template="Generate the detailed outline for a blog on the topic- {title}",
                              input_variables=['title'])
blog_prompt= PromptTemplate(template="Write a detail blog on the title {title} using the following outline {outline}",
                            input_variables=['title','outline'])
Evaluate_prompt= PromptTemplate( template=(
        "You are a strict blog evaluator.\n\n"
        "Blog Outline:\n{outline}\n\n"
        "Generated Blog:\n{blog}\n\n"
        "Evaluate how well the blog follows the outline, clarity, and completeness.\n"
        "Give a score from 1 to 10.\n"
        "Return ONLY the integer score."
    ),
    input_variables=["outline", "blog"]
)


In [11]:
outline_chain=outline_prompt | model
blog_chain=blog_prompt | model
Evaluate_chain= Evaluate_prompt| model

In [12]:
def create_outline(state: BLOGState)->BLOGState:
    response=outline_chain.invoke({
        'title': state['title']
    })
    state['outline']=response.content
    return state

In [13]:
def create_blog(state: BLOGState)->BLOGState:
    response=blog_chain.invoke({
            'title':state['title'],
            'outline':state['outline']
    })
    state['blog']=response.content
    return state

In [14]:
def evaluate_blog(state: BLOGState)->BLOGState:
    response=Evaluate_chain.invoke({
        'outline':state['outline'],
        'blog': state['blog']
    })
    state['score']=response.content
    return state

In [15]:
# Define graph
graph=StateGraph(BLOGState)
#Add node 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)
#Add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','evaluate_blog')
graph.add_edge('evaluate_blog',END)
# Compile
workflow=graph.compile()

In [19]:
#Execuate
final_state=workflow.invoke({
    'title':'rise in terriorism in pakistan'
})
print(final_state['blog'])



I. Introduction

The rise in terrorism in Pakistan has been a pressing issue that continues to threaten the stability and security of the country. It is crucial to address this issue in order to protect the lives of civilians, ensure economic development, and promote peace in the region. The current situation in Pakistan is characterized by various terrorist attacks, including those carried out by domestic and international terrorist groups.

II. Historical context of terrorism in Pakistan

The roots of terrorism in Pakistan can be traced back to the Soviet-Afghan War in the 1980s, when Pakistan supported the Afghan mujahideen against Soviet forces. This led to the rise of militant groups in Pakistan, who later turned against the Pakistani state. Major terrorist incidents in Pakistan include the attack on the Army Public School in Peshawar in 2014, the assassination of Benazir Bhutto in 2007, and numerous bombings and shootings targeting civilians and security forces. Terrorism has h